In [4]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import pandas as pd
import re

df = pd.read_csv('2023-11-07_Opros_pervokursnikov_3.csv')


for column in df.columns[3:]:
    # Удаление специальных символов из названия столбца
    cleaned_column_name = re.sub(r'\W+', '', column)
    
    tokenized_text = [word_tokenize(sentence.lower()) for sentence in df[column]]
    
    model = Word2Vec(sentences=tokenized_text, vector_size=100, window=5, min_count=1, workers=4)
    
    model.save(f'word2vec_{cleaned_column_name}.model')

In [13]:
print(word2vec_models)

{'какоеизутвержденийнаиболееблизкоописываетвашеотношениекучастиювнаучныхконференциях': <gensim.models.word2vec.Word2Vec object at 0x00000217600D6510>, 'вкакойтематикевыбыхотелиучаствоватьвнаучнойконференции': <gensim.models.word2vec.Word2Vec object at 0x000002175FFF6C50>, 'чтовасмоглобымотивироватьпринятьучастиевнаучнойконференции': <gensim.models.word2vec.Word2Vec object at 0x000002175F3393D0>, 'каквысчитаетенаскольковажнодляпервокурсниковучаствоватьвнаучныхконференциях': <gensim.models.word2vec.Word2Vec object at 0x000002176009EE10>, 'какиефакторымогутвасостановитьотучастиявнаучнойконференции': <gensim.models.word2vec.Word2Vec object at 0x0000021760165390>, 'вамбылобыинтересноучаствоватьвнаучнойконференциизаграницейвамбылобыинтересноучаствоватьвнаучнойконференциизаграницей': <gensim.models.word2vec.Word2Vec object at 0x000002176027EAD0>, 'еслибывыучаствоваливнаучнойконференциивыбыхотеливыступатьсдокладомилипростослушать': <gensim.models.word2vec.Word2Vec object at 0x000002176002BB90>

In [16]:
cleaned_column_names = [re.sub(r'\W+', '', column.lower()) for column in df.columns[3:]]
print(cleaned_column_names)

['какоеизутвержденийнаиболееблизкоописываетвашеотношениекучастиювнаучныхконференциях', 'вкакойтематикевыбыхотелиучаствоватьвнаучнойконференции', 'чтовасмоглобымотивироватьпринятьучастиевнаучнойконференции', 'каквысчитаетенаскольковажнодляпервокурсниковучаствоватьвнаучныхконференциях', 'какиефакторымогутвасостановитьотучастиявнаучнойконференции', 'вамбылобыинтересноучаствоватьвнаучнойконференциизаграницейвамбылобыинтересноучаствоватьвнаучнойконференциизаграницей', 'еслибывыучаствоваливнаучнойконференциивыбыхотеливыступатьсдокладомилипростослушать', 'выпредпочлибыучаствоватьвконференцияхвобластивашейспециализациииливдругихобластях', 'высчитаетечтоучастиевнаучныхконференцияхможетпомочьвамвучебе', 'какчастовыучаствовалившкольныхконференцияхивнеклассныхмероприятиях', 'чемвызанимаетесьвсвободноевремя', 'занимаетесьливысамообучением', 'покакойпричиневыпоступилинаданнуюспециальность', 'каквыотноситеськпубличномувыступлению', 'каквыотноситеськвозможностиобщенияисотрудничествасдругимистудентамив

In [37]:
def train_word2vec_model(text_column, model_name):
    cleaned_column_name = re.sub(r'\W+', '', model_name)
    
    tokenized_text = [word_tokenize(sentence.lower()) for sentence in text_column]
    
    # Проверка, что есть хотя бы одно непустое предложение
    if any(tokenized_text):
        model = Word2Vec(sentences=tokenized_text, vector_size=100, window=5, min_count=1, workers=4)
        model.save(f'word2vec_{cleaned_column_name}.model')
        print(f'Model trained and saved for column: {model_name}')
    else:
        print(f'No valid data for training model in column: {model_name}')

# Чтение DataFrame из CSV файла
df = pd.read_csv('2023-11-07_Opros_pervokursnikov_3.csv')

# Применение функции к каждой колонке
for column in df.columns[3:]:
    train_word2vec_model(df[column], column)

Model trained and saved for column: Какое из утверждений наиболее близко описывает ваше отношение к участию в научных конференциях?
Model trained and saved for column: В какой тематике вы бы хотели участвовать в научной конференции?
Model trained and saved for column: Что вас могло бы мотивировать принять участие в научной конференции?
Model trained and saved for column: Как вы считаете, насколько важно для первокурсников участвовать в научных конференциях?
Model trained and saved for column: Какие факторы могут вас остановить от участия в научной конференции?
Model trained and saved for column: Вам было бы интересно участвовать в научной конференции за границей?Вам было бы интересно участвовать в научной конференции за границей?
Model trained and saved for column: Если бы вы участвовали в научной конференции, вы бы хотели выступать с докладом или просто слушать?
Model trained and saved for column: Вы предпочли бы участвовать в конференциях в области вашей специализации или в других об

In [39]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Загрузите ваш датасет
df = pd.read_csv('2023-11-07_Opros_pervokursnikov_3.csv')

# Загрузите обученные модели Word2Vec
word2vec_models = {
    re.sub(r'\W+', '', col): Word2Vec.load(
        'word2vec_{}.model'.format(re.sub(r'\W+', '', col))
    ) for col in df.columns[3:]
}

# Функция для создания вектора ответа на основе Word2Vec модели
def get_response_vector(response, model):
    words = word_tokenize(response.lower())
    vectors = [model.wv[word] for word in words if word in model.wv]
    if vectors:
        return sum(vectors) / len(vectors)
    else:
        return [0] * model.vector_size

# Создайте векторы для каждого ответа в каждом столбце
response_vectors = {col: [get_response_vector(response, word2vec_models[col]) for response in df[col]] for col in df.columns[3:]}

# Кластеризация методом k-средних
kmeans = KMeans(n_clusters=5, random_state=42)
kmeans.fit(response_vectors['ваш_столбец'])

# Добавление меток кластеров в датасет
df['Cluster'] = kmeans.labels_

# Уменьшение размерности для визуализации (PCA)
pca = PCA(n_components=2)
reduced_features = pca.fit_transform(response_vectors['ваш_столбец'])
df[['PCA1', 'PCA2']] = reduced_features

# Визуализация кластеров
plt.scatter(df['PCA1'], df['PCA2'], c=df['Cluster'], cmap='viridis')
plt.title('Кластеризация ответов с использованием Word2Vec')
plt.show()


SyntaxError: unexpected character after line continuation character (1938751141.py, line 12)

In [24]:
# from gensim.models import Word2Vec
# from nltk.tokenize import word_tokenize
# from sklearn.metrics.pairwise import cosine_similarity
# import pandas as pd

# # Создайте пустой DataFrame для хранения результатов

# cleaned_column_names = [re.sub(r'\W+', '', column.lower()) for column in df.columns[3:]]
# result_df = pd.DataFrame(index=df.columns[3:], columns=df.columns[3:])
# word2vec_models = {cleaned_column: Word2Vec.load(f'word2vec_{cleaned_column}.model') for cleaned_column in cleaned_column_names}

# # Проход по всем парам колонок и вычисление сходства
# for col1 in cleaned_column_names:
#     for col2 in cleaned_column_names:
#         if col1 == col2:
#             result_df.at[col1, col2] = 1.0  # Схожесть с самим собой всегда 1
#         else:
#             # Сравнение предложений из соответствующих колонок
#             similarity = cosine_similarity([word2vec_models[col1].wv.get_vector(word) for word in word2vec_models[col1].wv.index_to_key],
#                                            [word2vec_models[col2].wv.get_vector(word) for word in word2vec_models[col2].wv.index_to_key])[0][0]
#             result_df.at[col1, col2] = similarity

# # Вывод результата
# print(result_df)


                                                   Какое из утверждений наиболее близко описывает ваше отношение к участию в научных конференциях?  \
Какое из утверждений наиболее близко описывает ...                                                NaN                                                
В какой тематике вы бы хотели участвовать в нау...                                                NaN                                                
Что вас могло бы мотивировать принять участие в...                                                NaN                                                
Как вы считаете, насколько важно для первокурсн...                                                NaN                                                
Какие факторы могут вас остановить от участия в...                                                NaN                                                
Вам было бы интересно участвовать в научной кон...                                                Na